In [3]:
#imports
import cv2
from imutils import paths
import numpy as np
import math

In [10]:
def generateDataset(filepath,strip_size,frames_number,strip_x_location,output_frame_length):
    
    frame_name_start = 0
    capture = cv2.VideoCapture(filepath)
    frameNr = 0

    output_frame_strips = math.ceil(output_frame_length/strip_size)         #Number of strips required to make one output frame

    print("Each output frame is a composite of ",output_frame_strips,"frames")
    print("Inital delay of ",output_frame_strips,"frames to form a full output image")



    # Create OpenCV objects
    bgsub = cv2.createBackgroundSubtractorMOG2()

    #Create ROI mask
    rectangle = np.zeros((512, 512), dtype="uint8")
    cv2.rectangle(rectangle, (150, 512), (350, 0), 255, -1)


    while (True):

        success, frame = capture.read()

        if success and (frameNr <frames_number + output_frame_strips ):

            backsub_frame = bgsub.apply(frame)  # apply MOG2 background subtraction    
            #skipped noise reduction
            th, bin_frame = cv2.threshold(backsub_frame, 128, 255, cv2.THRESH_BINARY)  #Binarise frame
            masked_frame = cv2.bitwise_and(rectangle, bin_frame)           # Apply mask to frame
            rotate_frame = cv2.rotate(masked_frame, cv2.ROTATE_90_CLOCKWISE)          # Rotate frame

            strip = rotate_frame[:, strip_x_location:strip_x_location +strip_size]

            if frameNr == 0 :
                final_frame = strip
            else :
                final_frame = np.concatenate((final_frame,strip),axis=1)
                
            if(len(final_frame[0]) == output_frame_length):         # When desired output length is reached, start removing tyhe first strip
                output_frameNr = frameNr - output_frame_strips + 1
                cv2.imwrite(f'data/images/bubbly/frame_{output_frameNr}.jpg', final_frame)
                final_frame = final_frame[:,strip_size:]
                
        else:
            break

        frameNr = frameNr+1
    capture.release()


In [40]:
path = 'data/regime_videos/'
regime_types = os.listdir(path)

print(regime_types)

path = path + regime_types[0]
video_names = os.listdir(path)
video_paths = [ (path +'/'+ i) for i in video_names ]


path = video_paths[1]
generateDataset(path,2,100,250,500)

['bubbly']
Each output frame is a composite of  250 frames
Inital delay of  250 frames to form a full output image


# Split Video into preprocessed frames

In [ ]:
capture = cv2.VideoCapture('data/regime_videos/bubbly/20210127_15.50_T=+5_m=5_x=0_q=AD_up_bubbly_512x512_mp4_C001H001S0001.mp4')
frameNr = 0

#Parameters
strip_size = 10            # Width of strip in pixels
frames_number =  100        # number of frames used
#alternate_frames = 1        # value of 1 uses all frames , 2 skips 1 frame 
strip_x_location = 250      # Horizontal loaction of strip start in pixels
output_frame_length = 500  # Size of Output frame in pixels


output_frame_strips = math.ceil(output_frame_length/strip_size)         #Number of strips required to make one output frame

print("Each output frame is a composite of ",output_frame_strips,"frames")
print("Inital delay of ",output_frame_strips,"frames to form a full output image")



# Create OpenCV objects
bgsub = cv2.createBackgroundSubtractorMOG2()

#Create ROI mask
rectangle = np.zeros((512, 512), dtype="uint8")
cv2.rectangle(rectangle, (150, 512), (350, 0), 255, -1)

 
while (True):
 
    success, frame = capture.read()
    
    if success and (frameNr <frames_number + output_frame_strips ):

        backsub_frame = bgsub.apply(frame)  # apply MOG2 background subtraction    
        #skipped noise reduction
        th, bin_frame = cv2.threshold(backsub_frame, 128, 255, cv2.THRESH_BINARY)  #Binarise frame
        masked_frame = cv2.bitwise_and(rectangle, bin_frame)           # Apply mask to frame
        rotate_frame = cv2.rotate(masked_frame, cv2.ROTATE_90_CLOCKWISE)          # Rotate frame

        strip = rotate_frame[:, strip_x_location:strip_x_location +strip_size]

        if frameNr == 0 :
            final_frame = strip
        else :
            final_frame = np.concatenate((final_frame,strip),axis=1)

        if(len(final_frame[0]) == output_frame_length):         # When desired output length is reached, start removing tyhe first strip
            output_frameNr = frameNr - output_frame_strips + 1
            cv2.imwrite(f'data/images/bubbly/frame_{output_frameNr}.jpg', final_frame)
            final_frame = final_frame[:,strip_size:]
            
    else:
        break
 
    frameNr = frameNr+1
capture.release()

